In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import pickle
from fair_loss import FairLoss
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import f1_score, balanced_accuracy_score,recall_score
import matplotlib.pyplot as plt


In [2]:
import torch
print(torch.__version__)


1.7.1


In [3]:

class LinReLU(nn.Module):
    """
    A linear layer followed by a ReLU activation layer.
    """

    def __init__(self, in_size, out_size):
        super(LinReLU, self).__init__()

        linear = nn.Linear(in_size, out_size)
        ReLU = nn.ReLU()
        # self.Dropout = nn.Dropout(0.25)
        self.layers = nn.Sequential(linear, ReLU)

    def reset_parameters(self):
        self.layers[0].reset_parameters()
        return self

    def forward(self, x):
        x = self.layers(x)
        return x


class FullyConnected(nn.Module):
    """
    A simple fully connected neural network with ReLU activations.
    """

    def __init__(self, input_size, layout):

        super(FullyConnected, self).__init__()
        layers = [nn.Flatten()]  # does not play any role, but makes the code neater
        prev_fc_size = input_size
        for i, fc_size in enumerate(layout):
            if i + 1 < len(layout):
                layers += [LinReLU(prev_fc_size, fc_size)]
            else:
                layers += [nn.Linear(prev_fc_size, fc_size)]
            prev_fc_size = fc_size
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        x = self.layers(x)
        return x



In [4]:
client_data_dir='../clients_data/processed_data/'

layout = [100, 100, 2]
batch_size = 32
num_epochs = 15
input_dim = 105
lr = 0.01

model = FullyConnected(input_dim, layout)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [6]:
client_models = [0,1,2,3]
client_models = [0]

In [11]:

def process_test_data(test_split, model, criterion):
    with open('../clients_data/processed_data/client_adult.pkl', 'rb') as f:
        test_data = pickle.load(f)
    
    total_length = sum(len(batch[0]) for batch in test_data)
    print(f"Total length of {test_split} test data: {total_length}")
    
    all_labels, all_predicted_classes = [], []
    model.eval()
    with torch.no_grad():
        val_running_loss = 0.0
        val_correct = 0
        val_total = 0
        
        for inputs, labels in test_data:
            outputs = model(inputs)
            val_loss = criterion(outputs, labels)
            val_running_loss += val_loss.item()
            _, predicted_classes = torch.max(outputs, dim=1)
            val_correct += (predicted_classes == labels).sum().item()
            val_total += labels.size(0)
            
            all_labels.extend(labels.cpu().numpy())
            all_predicted_classes.extend(predicted_classes.cpu().numpy())
            
        val_epoch_loss = val_running_loss / len(test_data)
        val_accuracy = val_correct / val_total
        
    all_labels = np.array(all_labels)
    all_predicted_classes = np.array(all_predicted_classes)
    
    f1 = f1_score(all_labels, all_predicted_classes, average="micro")
    balanced_acc = balanced_accuracy_score(all_labels, all_predicted_classes)
    tpr = recall_score(all_labels, all_predicted_classes, average="micro")
    
    return tpr, f1, balanced_acc, val_epoch_loss, val_accuracy

# Normal Traning

In [12]:
acc_normal_models={}
for model_name in client_models:
    print(f"------------------------ Processing model: {model_name}-------------------------")
    model.load_state_dict(torch.load(f'../clients_data/clients_trained_model/client{model_name}.pth'))
    tpr, f1, balanced_acc, val_epoch_loss, val_accuracy = process_test_data("client_adult", model, criterion)
    acc_normal_models[model_name]=val_accuracy
    print(f"model_name {model_name} - TPR: {tpr:.4f}, F1: {f1:.4f}, Balanced Accuracy: {balanced_acc:.4f}, Loss: {val_epoch_loss:.4f}, Accuracy: {val_accuracy:.4f}")
    

------------------------ Processing model: 0-------------------------
Total length of client_adult test data: 10000
model_name 0 - TPR: 0.7520, F1: 0.7520, Balanced Accuracy: 0.5000, Loss: 0.5982, Accuracy: 0.7520


In [13]:
acc_normal_models

{0: 0.752}

# Dp training

In [14]:
acc_dp_models={}
for model_name in client_models:
    print(f"------------------------ Processing model: {model_name}-------------------------")
    model.load_state_dict(torch.load(f'../clients_data/client_DP_trained_model/client{model_name}.pth'))
    tpr, f1, balanced_acc, val_epoch_loss, val_accuracy = process_test_data("client_adult", model, criterion)
    acc_dp_models[model_name]=val_accuracy
    print(f"model_name {model_name} - TPR: {tpr:.4f}, F1: {f1:.4f}, Balanced Accuracy: {balanced_acc:.4f}, Loss: {val_epoch_loss:.4f}, Accuracy: {val_accuracy:.4f}")
    

------------------------ Processing model: 0-------------------------
Total length of client_adult test data: 10000
model_name 0 - TPR: 0.7184, F1: 0.7184, Balanced Accuracy: 0.5421, Loss: 0.6715, Accuracy: 0.7184


In [15]:
acc_dp_models

{0: 0.7184}

# fair training

In [16]:
acc_fair_models={}
for model_name in client_models:
    print(f"------------------------ Processing model: {model_name}-------------------------")
    model.load_state_dict(torch.load(f'../clients_data/clients_fair_trained_model/client{model_name}.pth'))
    tpr, f1, balanced_acc, val_epoch_loss, val_accuracy = process_test_data("client_adult", model, criterion)
    acc_fair_models[model_name]=val_accuracy
    print(f"model_name {model_name} - TPR: {tpr:.4f}, F1: {f1:.4f}, Balanced Accuracy: {balanced_acc:.4f}, Loss: {val_epoch_loss:.4f}, Accuracy: {val_accuracy:.4f}")
    

------------------------ Processing model: 0-------------------------
Total length of client_adult test data: 10000
model_name 0 - TPR: 0.7288, F1: 0.7288, Balanced Accuracy: 0.5794, Loss: 0.6722, Accuracy: 0.7288


In [17]:
acc_fair_models

{0: 0.7288}

# fairDp Training

In [18]:
acc_fairDP_models={}
for model_name in client_models:
    print(f"------------------------ Processing model: {model_name}-------------------------")
    model.load_state_dict(torch.load(f'../clients_data/clients_DP_Fair_trained_model/client{model_name}.pth'))
    tpr, f1, balanced_acc, val_epoch_loss, val_accuracy = process_test_data("client_adult", model, criterion)
    acc_fairDP_models[model_name]=val_accuracy
    print(f"model_name {model_name} - TPR: {tpr:.4f}, F1: {f1:.4f}, Balanced Accuracy: {balanced_acc:.4f}, Loss: {val_epoch_loss:.4f}, Accuracy: {val_accuracy:.4f}")
    

------------------------ Processing model: 0-------------------------
Total length of client_adult test data: 10000
model_name 0 - TPR: 0.7520, F1: 0.7520, Balanced Accuracy: 0.5000, Loss: 0.5755, Accuracy: 0.7520


In [19]:
acc_fairDP_models

{0: 0.752}